# Capitulos 1 Introduction to Apache Spark's y 2: Download Apache Spark and Getting Started 

Iremos leyendo y realizando los ejemplos del capitulo 1 y 2 del libro, complementandolo con unos actividades sobre los mismos ejemplos. 

## Spark Session

La aplicación de Spark se controla a través de un controlador llamado SparkSession. La instancia SparkSession es la forma en que se ejecuta Spark y el código definido por el usuario. 

##### Siempre debemos iniciar una instancia SparkSession al principio. 

Creamos nuestra SparkSession en una variable spark, dandole como nombre Tema2, y verificamos la versión de spark que tiene:

In [1]:
import pyspark
import random
from pyspark.sql import SparkSession

# Creamos SparkSession (IMPORTANTE cambiar el nombre si estamos en otros script)
spark = SparkSession\
    .builder\
    .appName("LibroSpark_cap1y2")\
    .getOrCreate()

#Vemos versión de spark
print(spark.sparkContext.version)

3.0.1


# Ejercicios Capitulo 2

## A. Descargar el Quijote 
https://gist.github.com/jsdario/6d6c69398cb0c73111e49f1218960f79
#### Aplicar no solo count (para obtener el número de líneas) y show sino probar distintas sobrecargas del método show (con/sin truncate, indicando/sin indicar num de filas, etc) así como también los métodos, head, take, first (diferencias entre estos 3?)

In [2]:
# Cargamos el archivo el_quijote
Quijote = spark.read.text("./Datasets/el_quijote.txt")

In [3]:
# El metodo count() cuenta el número de lineas (2186)
Quijote.count()

2186

In [4]:
# El metodo first() devuelve solo la primera fila 
Quijote.first()

# Fijemonos que las muestra en tipo en una lista de tipo Row [Row(value='')]

Row(value='DON QUIJOTE DE LA MANCHA')

In [5]:
# El metodo show() devuelve el contenido del archivo por filas, en este caso 5, si no se indica numero, lo mostrará al completo.
Quijote.show(5)

+--------------------+
|               value|
+--------------------+
|DON QUIJOTE DE LA...|
|Miguel de Cervant...|
|                    |
|       PRIMERA PARTE|
|CAPÍTULO 1: Que ...|
+--------------------+
only showing top 5 rows



In [6]:
# Si nos fijamos, no muestra las filas al completo, añadiendo ... 
# Añadiendo al metodo show(truncate=False) Mostramos al completo las filas, por defecto este parámetro Truncate y es True
# Se puede poner directamente False y entenderá que es truncate=False
Quijote.show(5, False)

+----------------------------------------------------------------------------------------------+
|value                                                                                         |
+----------------------------------------------------------------------------------------------+
|DON QUIJOTE DE LA MANCHA                                                                      |
|Miguel de Cervantes Saavedra                                                                  |
|                                                                                              |
|PRIMERA PARTE                                                                                 |
|CAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha|
+----------------------------------------------------------------------------------------------+
only showing top 5 rows



In [7]:
# El metodo head() devuelve las primeras n filas, en este caso 5
Quijote.head(5)

# Fijemonos que las devuelve en tipo en una lista de tipo Row [Row(value='')] 

[Row(value='DON QUIJOTE DE LA MANCHA'),
 Row(value='Miguel de Cervantes Saavedra'),
 Row(value=''),
 Row(value='PRIMERA PARTE'),
 Row(value='CAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha')]

In [8]:
# El metodo take() devuelve las primeras n filas, en este caso 5. 
# head() y take () son metodos similares
Quijote.take(5)

[Row(value='DON QUIJOTE DE LA MANCHA'),
 Row(value='Miguel de Cervantes Saavedra'),
 Row(value=''),
 Row(value='PRIMERA PARTE'),
 Row(value='CAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha')]

A modo de conclusiones, las diferencias entre head, take y first son:
- First solo devuelve la primera liena del archivo
- Take y head devuelven las n primeras filas que le indiques en formato Row, siendo los dos metodos iguales
- Show muestra el archivo por filas, pudiendo indicarle el numero

## B. Del ejercicio del M&M aplicar:
#### B1 Hacer ejemplo del libro
#### B2 Otras operaciones de agregación como el Max con otro tipo de ordenamiento (descendiente).
#### B3 Hacer un ejercicio como el “where” de CA que aparece en el libro pero indicando más opciones de estados (p.e. NV, TX, CA, CO).
#### B4 Hacer un ejercicio donde se calculen en una misma operación el Max, Min, Avg, Count. Revisar el API (documentación) donde encontrarán este ejemplo:
ds.agg(max($"age"), avg($"salary"))
ds.groupBy().agg(max($"age"), avg($"salary"))
NOTA: $ es un alias de col()
iv.	Hacer también ejercicios en SQL creando tmpView



#### B1 Hacer ejemplo del libro

In [2]:
# Importaciones necesarias
from pyspark.sql.functions import *

In [4]:
# Cargamos DataFrame de M&M
mnm_df = spark.read.option("header", "true") \
          .option("inferSchema", "true") \
          .csv("./Datasets/mnm_dataset.csv")

# Mostramos con un print el numero de lineas
print("Total Rows = %d" % (mnm_df.count()))

# Inspeccionamos (show en modo tabla, take las coge como filas)
mnm_df.show(5)
# Vemos el schema del DF
mnm_df.printSchema()

Total Rows = 99999
+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
+-----+------+-----+
only showing top 5 rows

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)



 Conteo agregado de todos los colores y gruposPor estado y color en orden descentende.
 
 Hemos creado un nuevo DF a través de una consulta que selecciona las columnas "State", "Color" y "Count" agregadas por "State" y "Color" ordenando por el total de la columna "Count" en orden descendente de mayor a menor.
 
Count hace referencia al numero de veces que aparece un color estado, por lo que podremos ver los colores por estado

In [5]:
# B1 Total de colores agrupados por estado y color en orden descentende.
count_mnm_df = (mnm_df.select("State", "Color", "Count")
                      .groupBy("State", "Color")
                      .agg(sum("Count").alias("Total"))
                      .orderBy("Total", ascending=False))

# Mostramos con un print el numero de lineas
print("Total Rows = %d" % (count_mnm_df.count()))

count_mnm_df.show(5)

Total Rows = 60
+-----+------+------+
|State| Color| Total|
+-----+------+------+
|   CA|Yellow|100956|
|   WA| Green| 96486|
|   CA| Brown| 95762|
|   TX| Green| 95753|
|   TX|   Red| 95404|
+-----+------+------+
only showing top 5 rows



In [12]:
# Mostramos todas las lineas 
count_mnm_df.show(60, truncate=False)

+-----+------+------+
|State|Color |Total |
+-----+------+------+
|CA   |Yellow|100956|
|WA   |Green |96486 |
|CA   |Brown |95762 |
|TX   |Green |95753 |
|TX   |Red   |95404 |
|CO   |Yellow|95038 |
|NM   |Red   |94699 |
|OR   |Orange|94514 |
|WY   |Green |94339 |
|NV   |Orange|93929 |
|TX   |Yellow|93819 |
|CO   |Green |93724 |
|CO   |Brown |93692 |
|CA   |Green |93505 |
|NM   |Brown |93447 |
|CO   |Blue  |93412 |
|WA   |Red   |93332 |
|WA   |Brown |93082 |
|WA   |Yellow|92920 |
|NM   |Yellow|92747 |
|NV   |Brown |92478 |
|TX   |Orange|92315 |
|AZ   |Brown |92287 |
|AZ   |Green |91882 |
|WY   |Red   |91768 |
|AZ   |Orange|91684 |
|CA   |Red   |91527 |
|WA   |Orange|91521 |
|NV   |Yellow|91390 |
|UT   |Orange|91341 |
|NV   |Green |91331 |
|NM   |Orange|91251 |
|NM   |Green |91160 |
|WY   |Blue  |91002 |
|UT   |Red   |90995 |
|CO   |Orange|90971 |
|AZ   |Yellow|90946 |
|TX   |Brown |90736 |
|OR   |Blue  |90526 |
|CA   |Orange|90311 |
|OR   |Red   |90286 |
|NM   |Blue  |90150 |
|AZ   |Red

Filtramos el numero de colores por california (CA) en orden de mayor a menor.
 
Hemos creado un nuevo DF a través de una consulta que filtra los colores seleccionando todas las columnas con la condicion where de que sean del State de california, agrupadas por State y Color y ordenadas de mayor a menor por el total de colores.

Resumiendo vemos los colores y el total en el estado de california.

In [13]:
# B1.2 Filtramos por california (CA) Conteo agregado de todos los colores y gruposPor estado y color en orden descentende.
ca_count_mnm_df = (mnm_df
                .select("*")
                .where(mnm_df.State == 'CA')
                .groupBy("State", "Color")
                .agg(sum("Count").alias("Total"))
                .orderBy("Total", ascending=False))

# Mostramos con un print el numero de lineas
print("Total Rows = %d" % (ca_count_mnm_df.count()))

# Mostramos el resultado para las 10 primeras filas, que en realidad son 6
ca_count_mnm_df.show(truncate=False)

Total Rows = 6
+-----+------+------+
|State|Color |Total |
+-----+------+------+
|CA   |Yellow|100956|
|CA   |Brown |95762 |
|CA   |Green |93505 |
|CA   |Red   |91527 |
|CA   |Orange|90311 |
|CA   |Blue  |89123 |
+-----+------+------+



#### B2 Otras operaciones de agregación como el Max con otro tipo de ordenamiento (descendiente).

In [14]:
from pyspark.sql.functions import *

# B2 Agrupamos por State y color los valores minimos de colores (count) en cada estado en orden descendente 
min_mnm_df = (mnm_df.select("*")
                    .groupBy("State", "Color")
                    .agg(min("Count").alias('Minimos'))
                    .orderBy("Minimos", ascending=False))

# Mostramos con un print el numero de lineas
print("Total Rows = %d" % (min_mnm_df.count()))
min_mnm_df.show(5)

Total Rows = 60
+-----+------+-------+
|State| Color|Minimos|
+-----+------+-------+
|   CO|  Blue|     10|
|   NM|Orange|     10|
|   NM| Green|     10|
|   UT|  Blue|     10|
|   AZ|Orange|     10|
+-----+------+-------+
only showing top 5 rows



#### B3 Hacer un ejercicio como el “where” de CA que aparece en el libro pero indicando más opciones de estados (p.e. NV, TX, CA, CO).

In [6]:
# Los operadores booleanos AND, OR se escriben en spark como & y | Respectivamente (primer .where)
# Se puede usar el operador in para varias condiciones englobandolo todo entre "" (segundo where)
# en la condicion where() englobamos las columnas con el valor de la condicion y separando por el operador 

# B3 Filtramos por los estados (NV, TX, CA, CO) el Total de colores agrupados por estado y color en orden descentende.
B3_mnm_df = (mnm_df.select("*")
                #.where( (mnm_df.State == 'CA') | (mnm_df.State == 'NV') | (mnm_df.State == 'CA') | (mnm_df.State == 'CO') )
                .where("State in ('NV', 'TX', 'CA', 'CO')")
                .groupBy("State", "Color")
                .agg(sum("Count").alias("Total"))
                .orderBy("Total", ascending=False))
B3_mnm_df.show(5)

# Se puede sustituir la funcion where por filter obteniendo el mismo resultado. son funciones similares

+-----+------+------+
|State| Color| Total|
+-----+------+------+
|   CA|Yellow|100956|
|   CA| Brown| 95762|
|   TX| Green| 95753|
|   TX|   Red| 95404|
|   CO|Yellow| 95038|
+-----+------+------+
only showing top 5 rows



#### B4 Hacer un ejercicio donde se calculen en una misma operación el Max, Min, Avg, Count. Revisar el API (documentación) donde encontrarán este ejemplo:

ds.agg(max( $ "age"), avg( $ "salary"))

ds.groupBy().agg(max( $ "age"), avg( $ "salary"))

NOTA: $ es un alias de col()


In [16]:
# B4 Select de todas las columnas agrupado por estado y color y calculando con agg minimos, maximos, media y numero de colores
agg_mnm_df = (mnm_df.select("State", "Color", "Count")
              .groupBy("State", "Color")
              .agg( max("Count").alias("Maximos_count"), 
                    min("Count").alias("Minimos_coutn"),
                    avg("Count").alias("AVG_count"), 
                    count("Count").alias("Total_count") )

)

agg_mnm_df.show(5)

+-----+------+-------------+-------------+------------------+-----------+
|State| Color|Maximos_count|Minimos_coutn|         AVG_count|Total_count|
+-----+------+-------------+-------------+------------------+-----------+
|   WY| Green|          100|           10|55.657227138643066|       1695|
|   NV|   Red|          100|           10|  55.4944099378882|       1610|
|   UT|  Blue|          100|           10|54.366767371601206|       1655|
|   WA|Orange|          100|           10|55.199638118214715|       1658|
|   NM| Green|          100|           10|  54.1973840665874|       1682|
+-----+------+-------------+-------------+------------------+-----------+
only showing top 5 rows



#### B5 Hacer también ejercicios en SQL creando tmpView

In [17]:
# Creamos vista temporal del DF del ejercicio anterior (agg_mnm_df)
agg_mnm_df.createOrReplaceTempView("agg_view")

# Así podremos hacer consultas puras en sql lanzando el sqlContext, como por ejemplo:
results = spark.sql("SELECT * FROM agg_view LIMIT 5")
results.show()

+-----+------+-------------+-------------+------------------+-----------+
|State| Color|Maximos_count|Minimos_coutn|         AVG_count|Total_count|
+-----+------+-------------+-------------+------------------+-----------+
|   WY| Green|          100|           10|55.657227138643066|       1695|
|   NV|   Red|          100|           10|  55.4944099378882|       1610|
|   UT|  Blue|          100|           10|54.366767371601206|       1655|
|   WA|Orange|          100|           10|55.199638118214715|       1658|
|   NM| Green|          100|           10|  54.1973840665874|       1682|
+-----+------+-------------+-------------+------------------+-----------+



Ahora, repetiremos todas las consultas agregadas realizadas anteriormente pero en formato SQL puro. Crearemos una vista temporal con el DF original y iremos haciendo las querys en SQL

El resultado y el procesamiento por parte de Spark será igual indistintamente entre el trabajo con Dataframes y SQL puro.

In [18]:
# Creamos vista temporal del DF mnm_df
mnm_df.createOrReplaceTempView("mnm_df_view")

In [19]:
# B1 Total de colores agrupados por estado y color en orden descentende. (SQLpuro)
results_total_color = spark.sql("SELECT State, Color, sum(Count) AS Total \
                                 FROM mnm_df_view \
                                 GROUP BY State, Color \
                                 ORDER BY Total DESC  \
                                 LIMIT 5") \
                           .show()

+-----+------+------+
|State| Color| Total|
+-----+------+------+
|   CA|Yellow|100956|
|   WA| Green| 96486|
|   CA| Brown| 95762|
|   TX| Green| 95753|
|   TX|   Red| 95404|
+-----+------+------+



In [20]:
# B1.2 Filtramos por california (CA) Conteo agregado de todos los colores y gruposPor estado y color en orden descentende
# (SQL puro).

results_total_color_ca = spark.sql("SELECT State, Color, sum(Count) AS Total \
                                 FROM mnm_df_view \
                                 WHERE State = 'CA' \
                                 GROUP BY State, Color \
                                 ORDER BY Total DESC  \
                                 LIMIT 5") \
                              .show()


+-----+------+------+
|State| Color| Total|
+-----+------+------+
|   CA|Yellow|100956|
|   CA| Brown| 95762|
|   CA| Green| 93505|
|   CA|   Red| 91527|
|   CA|Orange| 90311|
+-----+------+------+



In [21]:
# B2 Agrupamos por State y color los valores minimos de colores (count) en cada estado en orden descendente 
# (SQL puro).
results_min_color =  spark.sql("SELECT State, Color, min(Count) AS Minimos \
                                 FROM mnm_df_view \
                                 GROUP BY State, Color \
                                 ORDER BY Minimos DESC  \
                                 LIMIT 5") \
                          .show()

+-----+------+-------+
|State| Color|Minimos|
+-----+------+-------+
|   WY| Green|     10|
|   NV|   Red|     10|
|   UT|  Blue|     10|
|   WA|Orange|     10|
|   NM| Green|     10|
+-----+------+-------+



In [22]:
# B3 Filtramos por los estados (NV, TX, CA, CO) el Total de colores agrupados por estado y color en orden descentende.
# SQL puro
results_B3 = spark.sql("SELECT State, Color, sum(Count) AS Total \
                        FROM mnm_df_view \
                        WHERE State IN ('NV', 'TX', 'CA', 'CO')\
                        GROUP BY State, Color \
                        ORDER BY Total DESC") \
                  .show(5)

# Podemos mostrar 5 columnas con show() o con LIMIT, pudiendo combinar los dos lenguajes (DF y SQL)

+-----+------+------+
|State| Color| Total|
+-----+------+------+
|   CA|Yellow|100956|
|   CA| Brown| 95762|
|   TX| Green| 95753|
|   TX|   Red| 95404|
|   CO|Yellow| 95038|
+-----+------+------+
only showing top 5 rows



In [23]:
# B4 Select de todas las columnas agrupado por estado y color y calculando con agg minimos, maximos, media y numero de colores
# SQL puro
results_agg_B4 = spark.sql("SELECT State, Color, min(Count) AS Minimos, max(Count) AS Maximos, avg(Count) AS AVG, sum(Count) AS Total\
                                 FROM mnm_df_view \
                                 WHERE State IN ('NV', 'TX', 'CA', 'CO')\
                                 GROUP BY State, Color \
                                 ORDER BY Total DESC") \
                      .show(5)

+-----+------+-------+-------+------------------+------+
|State| Color|Minimos|Maximos|               AVG| Total|
+-----+------+-------+-------+------------------+------+
|   CA|Yellow|     10|    100|  55.8693967902601|100956|
|   CA| Brown|     10|    100|55.740395809080326| 95762|
|   TX| Green|     10|    100| 55.12550374208405| 95753|
|   TX|   Red|     10|    100|55.306666666666665| 95404|
|   CO|Yellow|     10|    100| 55.22254503195816| 95038|
+-----+------+-------+-------+------------------+------+
only showing top 5 rows



In [24]:
spark.stop()